In [ ]:
#for converstion to jpg

# import os
# import rawpy
# from PIL import Image

# # Function to convert .cr3 files to .jpg
# def convert_cr3_to_jpg(input_folder, output_folder):
#     for root, _, files in os.walk(input_folder):
#         for file in files:
#             if file.endswith(".CR3"):
#                 try:
#                     # Input and output paths
#                     input_path = os.path.join(root, file)
#                     output_path = os.path.join(output_folder, os.path.relpath(root, input_folder), file.replace(".CR3", ".jpg"))

#                     # Create output folder if it doesn't exist
#                     os.makedirs(os.path.dirname(output_path), exist_ok=True)

#                     # Convert .cr3 to .jpg
#                     with rawpy.imread(input_path) as raw:
#                         rgb = raw.postprocess()
#                         image = Image.fromarray(rgb)
#                         image.save(output_path, "JPEG")
#                     print(f"Converted {input_path} to {output_path}")
#                 except Exception as e:
#                     print(f"Error converting {input_path}: {e}")

# # Input and output folders
# input_folder = "dataset"
# output_folder = "changed_dataset"

# # Convert .cr3 to .jpg
# convert_cr3_to_jpg(input_folder, output_folder)


In [10]:
import os
import numpy as np
from PIL import Image
import imgaug.augmenters as iaa

# Define augmentation pipeline
seq = iaa.Sequential([
    iaa.Fliplr(0.5),   # horizontally flip 50% of the images
    iaa.Flipud(0.5),   # vertically flip 50% of the images
    iaa.Affine(rotate=(-10, 10)),  # rotate by -10 to +10 degrees
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # add gaussian noise
    iaa.GaussianBlur(sigma=(0, 3.0)),  # blur images with a sigma of 0 to 3.0
    iaa.Grayscale(alpha=(0.0, 1.0)),  # convert images to grayscale
    iaa.Sometimes(0.5, iaa.ChannelShuffle(1.0)),  # shuffle channels
    iaa.Sometimes(0.5, iaa.Invert(1.0)),  # invert pixel values
    iaa.Sometimes(0.5, iaa.LinearContrast((0.5, 2.0))),  # linear contrast adjustment
    iaa.Sometimes(0.5, iaa.HistogramEqualization()),  # histogram equalization
    iaa.Sometimes(0.5, iaa.GammaContrast((0.5, 2.0))),  # gamma contrast adjustment
    iaa.Sometimes(0.5, iaa.CLAHE()),  # contrast limited adaptive histogram equalization
    iaa.Sometimes(0.5, iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV")),  # RGB to HSV
    iaa.Sometimes(0.5, iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="Lab")), # RGB to LAB
    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 1.0))),  # gaussian blur
    iaa.Sometimes(0.5, iaa.MotionBlur(k=15, angle=[-45, 45])),  # motion blur
    iaa.Sometimes(0.5, iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 2.0))),  # sharpen images
    iaa.Sometimes(0.5, iaa.AddToBrightness((-50, 50))),  # change brightness of images
    iaa.Sometimes(0.5, iaa.AddToHueAndSaturation((-20, 20))),  # change hue and saturation
    iaa.Sometimes(0.5, iaa.MultiplyHueAndSaturation((0.5, 1.5))),  # multiply hue and saturation
    iaa.Sometimes(0.5, iaa.LinearContrast((0.5, 2.0))),  # linear contrast adjustment
    iaa.Sometimes(0.5, iaa.AllChannelsCLAHE()),  # contrast limited adaptive histogram equalization for all channels
    iaa.Sometimes(0.5, iaa.AllChannelsHistogramEqualization()),  # histogram equalization for all channels
    iaa.Sometimes(0.5, iaa.AddToHue((-50, 50))),  # change hue of images
    iaa.Sometimes(0.5, iaa.SigmoidContrast(gain=(3, 10), cutoff=(0.4, 0.6))),  # sigmoid contrast adjustment
    iaa.Sometimes(0.5, iaa.FastSnowyLandscape(lightness_threshold=(128, 255))),  # snowy landscape effect
    iaa.Sometimes(0.5, iaa.Superpixels(p_replace=(0.1, 1.0), n_segments=(16, 128))),  # superpixels
    iaa.Sometimes(0.5, iaa.Cartoon()),  # cartoon effect
    iaa.Sometimes(0.5, iaa.Emboss(alpha=(0.0, 1.0), strength=(0.5, 1.5))),  # emboss effect
    iaa.Sometimes(0.5, iaa.AverageBlur(k=(2, 7))),  # average blur
    iaa.Sometimes(0.5, iaa.MedianBlur(k=(3, 11))),  # median blur
    iaa.Sometimes(0.5, iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)),  # elastic transformation
    iaa.Sometimes(0.5, iaa.PerspectiveTransform(scale=(0.01, 0.1))),  # perspective transformation
    iaa.Sometimes(0.5, iaa.PiecewiseAffine(scale=(0.01, 0.05))),  # piecewise affine transformation
    iaa.Sometimes(0.5, iaa.CoarseDropout(0.02, size_percent=0.1)),  # coarse dropout
    iaa.Dropout(0.02),
    iaa.Sometimes(0.5, iaa.Cutout(nb_iterations=(1, 3), size=0.2, squared=False)),  # cutout
    iaa.AddToBrightness((-50, 50)), # Adjust brightness by -50 to 50
    iaa.LinearContrast((0.5, 2.0)), # Adjust contrast by a factor between 0.5 and 2.0
    iaa.AverageBlur(k=(2, 7)), # Apply average blur with a kernel size between 2 and 7
    iaa.Sometimes(0.5, iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 2.0))),  # sharpen
])

# Function to perform image augmentation
def augment_images(input_folder, output_folder, target_images):
    total_images = sum(len(files) for _, _, files in os.walk(input_folder))
    augmentation_factor = target_images / total_images

    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith(".jpg"):
                try:
                    # Input and output paths
                    input_path = os.path.join(root, file)
                    output_path = os.path.join(output_folder, os.path.relpath(root, input_folder), file)

                    # Load image
                    image = Image.open(input_path)
                    image = np.array(image)

               
                    # Augment image
                    for i in range(int(augmentation_factor)):
                        augmented_images = seq(images=[image])
                        for j, augmented_image in enumerate(augmented_images):
                            augmented_image = Image.fromarray(augmented_image)
                            augmented_image.save(os.path.join(output_path, f"{file.split('.')[0]}_aug_{i*len(augmented_images) + j}.jpg"))
                    print(f"Augmented {input_path}")
                except Exception as e:
                    print(f"Error augmenting {input_path}: {e}")

# Input and output folders
input_folder = "code\changed_dataset"
output_folder = "code\augmented_photos"

# Perform image augmentation
target_images = 2000
augment_images(input_folder, output_folder, target_images)


<>:80: SyntaxWarning: invalid escape sequence '\c'
<>:80: SyntaxWarning: invalid escape sequence '\c'
C:\Users\BotCat\AppData\Local\Temp\ipykernel_9464\4167042945.py:80: SyntaxWarning: invalid escape sequence '\c'
  input_folder = "code\changed_dataset"
C:\Users\BotCat\AppData\Local\Temp\ipykernel_9464\4167042945.py:80: SyntaxWarning: invalid escape sequence '\c'
  input_folder = "code\changed_dataset"


ZeroDivisionError: division by zero

In [ ]:
Error augmenting changed_dataset\0.052506\IMG_2448.jpg: [Errno 2] No such file or directory: 'augmented_photos\\0.052506\\IMG_2448.jpg\\IMG_2448_aug_0.jpg'